Import data

In [118]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime as dt
from sklearn import * 
% matplotlib inline

data = pd.read_csv(r'C:\Users\anhem44\Desktop\Capstone 2\properties_2016.csv',index_col='parcelid')
y_data = pd.read_csv(r'C:\Users\anhem44\Desktop\Capstone 2\train_2016_v2.csv',index_col='parcelid')

C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Divide data into clean data, categorical & continuous data with NaNs that need to be predicted

In [119]:
#Step 1
def clean_x(x) :
    #seperate data into frames based off of type
    
    cat_data = x[['architecturalstyletypeid','buildingqualitytypeid','numberofstories','calculatedbathnbr']]

    cont_data = x[['structuretaxvaluedollarcnt','censustractandblock','lotsizesquarefeet','finishedsquarefeet50',
                  'finishedsquarefeet15','finishedsquarefeet12','calculatedfinishedsquarefeet','finishedfloor1squarefeet']]

    others = x.drop(x[['architecturalstyletypeid','buildingqualitytypeid','regionidneighborhood','numberofstories',
                         'calculatedbathnbr','structuretaxvaluedollarcnt','censustractandblock','lotsizesquarefeet',
                         'finishedsquarefeet50','finishedsquarefeet15','finishedsquarefeet12','calculatedfinishedsquarefeet',
                         'finishedfloor1squarefeet']], axis = 1)

    others.fillna(0, inplace=True)
    others.head()

    data_list = others.select_dtypes(include=['object']) # list all non-numeric rows
    numeric_only = others._get_numeric_data() #select only numeric rows

    for column in data_list : #make sure all non-numerics are strings
        data_list[column] = data_list[column].astype(str)

    data_list = data_list.apply(preprocessing.LabelEncoder().fit_transform) #encode all nun-numerics using labels
    numeric_only = numeric_only.apply(preprocessing.LabelEncoder().fit_transform)
    data_clean = pd.concat([numeric_only, data_list], axis=1,join_axes=[data.index]) #re-join numeric and non-numeric
    
    return cat_data, cont_data, data_clean

Predict categorical variables

In [120]:
#step 2
def cat_nan(cat_data,data_clean):
    #use list of names for for loop
    cat_list = list(cat_data) #turn column names into a list
    
    #dfs function will fill and return
    cat_df = pd.DataFrame(index=data.index) #will fill this df w/best prediction answer in a column
    cat_nan_df = pd.DataFrame(columns=['KNN_mean','LGR_mean','RF_mean','best']) #df that will give the score and best 
    
    for i in cat_list :
        # 1- select only those cells w/values for training set
        df_c = pd.concat([data_clean, cat_data[i]],axis=1,join_axes=[data_clean.index]) #join cat data to clean data
        
        df_n_null = df_c[i].notnull() #data that is not null for training set
        df_n_null = df_c[df_n_null]
        df_null = df_c[i].isnull() #data that is null for prediction set
        df_null = df_c[df_null]
        
        y = pd.DataFrame(df_n_null[i]) #target as df
        x = df_n_null.drop(i,axis=1) #remove target from testing set as df
        
        y = y.apply(preprocessing.LabelEncoder().fit_transform) #preproccess target
        
        # 2- Use kfolds for test
        kfold = cross_validation.KFold(len(x),n_folds=5, random_state=42)
        
        # 3- run different tests
        #KNN on test data
        KNN_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test] 
            KNN = neighbors.KNeighborsClassifier() 
            KNN.fit(x_train,y_train)
            KNN_score = KNN.score(x_test,y_test)
            KNN_pred_error.append(KNN_score)
        KNN_mean = np.mean(KNN_pred_error) #mean of each score
        
        #logistic regression on test data
        LGR_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
            LGR = linear_model.LogisticRegression()
            LGR.fit(x_train,y_train)
            LGR_score = LGR.score(x_test,y_test)
            LGR_pred_error.append(LGR_score)
        LGR_mean = np.mean(LGR_pred_error)
        
        #random forest
        RF_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
            RF = ensemble.RandomForestClassifier()
            RF.fit(x_train,y_train)
            RF_score = RF.score(x_test,y_test)
            RF_pred_error.append(RF_score)
        RF_mean = np.mean(RF_pred_error)
        
        # 4-calculate best predictor
        
        best = str() #will tag best predictor
        if (KNN_mean != LGR_mean) & (KNN_mean != RF_mean) & (LGR_mean != RF_mean) :
            if (KNN_mean > LGR_mean) & (KNN_mean > RF_mean) :
                best = 'KNN' 
            elif (LGR_mean > KNN_mean) & (LGR_mean > RF_mean) :
                best = 'LGR'
            elif (RF_mean > LGR_mean) & (RF_mean > KNN_mean) :
                best = 'RF'
        elif KNN_mean == LGR_mean :
            best = 'LGR' 
        elif KNN_mean == RF_mean :
            best = 'KNN'
        else :
            best = 'LGR'
        
        # add to df
        cat_nan_df.loc[i] = [KNN_mean,LGR_mean,RF_mean,best] #add all means and best to df
        
        # 5- use best predictor to predict missing values for predictor set
        x_null = df_null.drop(i,axis=1)
        
        if best == 'KNN':
            KNN = neighbors.KNeighborsClassifier()
            KNN.fit(x,y)
            prediction = KNN.predict(x_null) 
        elif best == 'LGR':
            LGR = linear_model.LogisticRegression()
            LGR.fit(x,y)
            prediction = LGR.predict(x_null)
        elif best == 'RF':
            RF = ensemble.RandomForestClassifier()
            RF.fit(x,y)
            prediction = RF.predict(x_null)
        
        # 6-insert prediction into data
        predicton_df = pd.DataFrame()
        x_total = pd.DataFrame()
        
        predicton_df[i] = prediction
        predicton_df.index = x_null.index #re-add index to prediction
        
        x_total = pd.concat([predicton_df,y]) #combine prediction and test set
        cat_df = pd.concat([cat_df, x_total], axis=1) #add complete column to df
        
    return cat_nan_df, cat_df

Use the same procedure for continuous data

In [121]:
#step 3

#select only those cells w/values
def lin_nan(cont_data,data_clean):
    #use list of names for for loop
    lin_list = list(cont_data)
    
    #dfs function will fill and return
    lin_df = pd.DataFrame(index=data.index)
    lin_nan_df = pd.DataFrame(columns=['LR_mean','SV_mean','RF_mean','best'])
    
    for i in lin_list :
        df_c = pd.concat([data_clean, cont_data[i]],axis=1,join_axes=[data_clean.index])
        
        df_n_null = df_c[i].notnull()
        df_n_null = df_c[df_n_null]
        df_null = df_c[i].isnull()
        df_null = df_c[df_null]
        
        y = pd.DataFrame(df_n_null[i])
        x = df_n_null.drop(i,axis=1)
        
        y = y.apply(preprocessing.LabelEncoder().fit_transform)
        
        #Use kfolds for test
        kfold = cross_validation.KFold(len(x),n_folds=5, random_state=42)
        
        #linear regression
        LR_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
            LR = linear_model.LinearRegression()
            LR.fit(x_train,y_train)
            LR_score = LR.score(x_test,y_test)
            LR_pred_error.append(LR_score)
        LR_mean = np.mean(LR_pred_error)
        
        #SVM
        SV_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
            SV = svm.LinearSVR()
            SV.fit(x_train,y_train)
            SV_score = SV.score(x_test,y_test)
            SV_pred_error.append(SV_score)
        SV_mean = np.mean(SV_pred_error)
        
        #random forest
        RF_pred_error = []
        for train, test in kfold :
            x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
            RF = ensemble.RandomForestRegressor()
            RF.fit(x_train,y_train)
            RF_score = RF.score(x_test,y_test)
            RF_pred_error.append(RF_score)
        RF_mean = np.mean(RF_pred_error)
        
        #calculate best predictor
        best = str()
        if (LR_mean != SV_mean) & (LR_mean != RF_mean) & (SV_mean != RF_mean) :
            if (LR_mean > SV_mean) & (LR_mean > RF_mean) :
                best = 'LR'
            elif (SV_mean > LR_mean) & (SV_mean > RF_mean) :
                best = 'SV'
            elif (RF_mean > SV_mean) & (RF_mean > LR_mean) :
                best = 'RF'
        elif LR_mean == SV_mean :
            best = 'SV'
        elif LR_mean == RF_mean :
            best = 'KNN'
        else :
            best = 'SV'
        
        #add to df
        lin_nan_df.loc[i] = [LR_mean,SV_mean,RF_mean,best]
        
        #use best predictor to predict missing values
        x_null = df_null.drop(i,axis=1)
        
        if best == 'LR':
            LR = linear_model.LinearRegression()
            LR.fit(x,y)
            prediction = LR.predict(x_null) 
        elif best == 'SV':
            SV = svm.LinearSVR()
            SV.fit(x,y)
            prediction = LGR.predict(x_null)
        elif best == 'RF':
            RF = ensemble.RandomForestRegressor()
            RF.fit(x,y)
            prediction = RF.predict(x_null)
        
        #insert prediction into data
        predicton_df = pd.DataFrame()
        x_total = pd.DataFrame()
        
        predicton_df[i] = prediction
        predicton_df.index = x_null.index
        x_total = pd.concat([predicton_df,y])
        lin_df = pd.concat([lin_df, x_total], axis=1)
        
    return lin_nan_df, lin_df

Combine data into one frame and split out training data

In [122]:
#step 4
def final_comb(data_clean,lin_pred,cat_pred,y):    
    final_df = pd.concat([data_clean,lin_pred, cat_pred], axis=1)
    
    train_data = pd.concat([y, final_df], axis=1,join_axes=[y.index])
    train_data['transactiondate'] = pd.to_datetime(train_data['transactiondate'])
    train_data['year'] = train_data['transactiondate'].dt.year
    train_data['month']  = train_data['transactiondate'].dt.month
    train_data['parcelid'] = train_data.index
    train_data.set_index(['parcelid','transactiondate'], inplace=True)
    final_df['month']  = 0
    final_df['year']  = 0
    return final_df, train_data

Do final prediction

In [123]:
#step 5

#select only those cells w/values
def fin_analysis(train_data,final_df):
    fin_df = pd.DataFrame(index=data.index)
    fin_nan_df = pd.DataFrame(columns=['LR_mean','SV_mean','RF_mean','best'])
    
    t_data = train_data.copy()
    y = pd.DataFrame(train_data['logerror'])
    x = train_data.drop(train_data[['logerror']], axis=1)
    
    #Use kfolds for test
    kfold = cross_validation.KFold(len(x),n_folds=5, random_state=42)
        
        #linear regression
    LR_pred_error = []
    for train, test in kfold :
        x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
        LR = linear_model.LinearRegression()
        LR.fit(x_train,y_train)
        LR_score = LR.score(x_test,y_test)
        LR_pred_error.append(LR_score)
    LR_mean = np.mean(LR_pred_error)
        
        #SVM
    SV_pred_error = []
    for train, test in kfold :
        x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
        SV = svm.LinearSVR()
        x_train = preprocessing.StandardScaler().fit_transform(x_train,y_train)
        SV.fit(x_train,y_train)
        SV_score = SV.score(x_test,y_test)
        SV_pred_error.append(SV_score)
    SV_mean = np.mean(SV_pred_error)
        
        #random forest
    RF_pred_error = []
    for train, test in kfold :
        x_train, x_test, y_train, y_test = x.iloc[train], x.iloc[test], y.iloc[train], y.iloc[test]
        RF = ensemble.RandomForestRegressor()
        RF.fit(x_train,y_train)
        RF_score = RF.score(x_test,y_test)
        RF_pred_error.append(RF_score)
    RF_mean = np.mean(RF_pred_error)
        
        #calculate best predictor
        
                        #best 2 out of three
    best = str()
    if (LR_mean != SV_mean) & (LR_mean != RF_mean) & (SV_mean != RF_mean) :
        if (LR_mean > SV_mean) & (LR_mean > RF_mean) :
            best = 'LR'
        elif (SV_mean > LR_mean) & (SV_mean > RF_mean) :
            best = 'SV'
        elif (RF_mean > SV_mean) & (RF_mean > LR_mean) :
            best = 'RF'
    elif LR_mean == SV_mean :
        best = 'SV'
    elif LR_mean == RF_mean :
        best = 'KNN'
    else :
        best = 'SV'
        
        #add to df
    fin_nan_df.loc['FINAL'] = [LR_mean,SV_mean,RF_mean,best]
        
    dates = [201610,201611,201612,201710,201711,201712]
    predicton_df = pd.DataFrame(index=final_df.index)
    #predicton_df.index = final_df.index
    
    x = x.reset_index(level=1, drop=True)
    #final_df = final_df.drop(final_df[['regionidneighborhood']], axis = 1)
    
    for date in dates :
        final_df['month'] = date
        if best == 'LR':
            LR = linear_model.LinearRegression()
            LR.fit(x,y)
            prediction = LR.predict(final_df)
            predicton_df[date] = prediction
        elif best == 'SV':
            SV = svm.LinearSVR()
            SV.fit(x,y)
            prediction = LGR.predict(x_null)
            prediction = LR.predict(final_df)
            predicton_df[date] = prediction
        elif best == 'RF':
            RF = ensemble.RandomForestRegressor()
            RF.fit(x,y)
            prediction = RF.predict(x_null)
            prediction = LR.predict(final_df)
            predicton_df[date] = prediction
    
    return fin_nan_df, predicton_df

Run all formulas

In [ ]:
cat_data, cont_data, data_clean = clean_x(data)
cat_info, cat_pred = cat_nan(cat_data,data_clean)
lin_info, lin_pred = lin_nan(cont_data,data_clean)
final_predict, final_train = final_comb(data_clean,lin_pred,cat_pred,y_data)

final_predict.to_csv(r'C:\Users\anhem44\Desktop\Capstone 2\final_df_2.csv')
final_train.to_csv(r'C:\Users\anhem44\Desktop\Capstone 2\final_train_data_2.csv')

pred_df, fin_info = fin_analysis(final_train,final_predict)

C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\anhem44\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5

In [ ]:
pred_df

In [ ]:
fin_df